In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv("C:/Users/Siddhi/Downloads/Fraud.csv")

DATA PREPROCESSING 

In [3]:
df1=df
df.tail(20)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
6362600,742,TRANSFER,652993.91,C40604503,652993.91,0.0,C1166857907,0.00,0.00,1,0
6362601,742,CASH_OUT,652993.91,C1614818636,652993.91,0.0,C362803701,0.00,652993.91,1,0
6362602,742,TRANSFER,1819543.69,C2089752665,1819543.69,0.0,C112833674,0.00,0.00,1,0
6362603,742,CASH_OUT,1819543.69,C1039979813,1819543.69,0.0,C2078394828,0.00,1819543.69,1,0
6362604,742,TRANSFER,54652.46,C1674778854,54652.46,0.0,C1930074465,0.00,0.00,1,0
6362605,742,CASH_OUT,54652.46,C43545501,54652.46,0.0,C830041824,0.00,54652.46,1,0
6362606,742,TRANSFER,303846.74,C959102961,303846.74,0.0,C114421319,0.00,0.00,1,0
6362607,742,CASH_OUT,303846.74,C1148860488,303846.74,0.0,C846260566,343660.89,647507.63,1,0
6362608,742,TRANSFER,258355.42,C1226129332,258355.42,0.0,C1744173808,0.00,0.00,1,0
6362609,742,CASH_OUT,258355.42,C1113162093,258355.42,0.0,C797688696,25176.67,283532.09,1,0


In [4]:
df.dtypes

step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

In [5]:
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

Checking for outliers using box-plot

In [7]:
# l=df.columns
# for x in l:
#     if df[x].dtype=="int64" or df[x].dtype=="float64":
#         plt.boxplot(df[x])
#         plt.title(x)
#         plt.show()
def detect_outliers(series):
    q1=series.quantile(0.25)
    q3=series.quantile(0.75)
    iqr=q3-q1
    lower_bound=q1-1.5*iqr
    upper_bound=q3+1.5*iqr
    outliers=(series<lower_bound)|(series>upper_bound)
    return outliers.sum()
for x in df.columns:
    if df[x].dtype=="int64" or df[x].dtype=="float64":
        num_outliers = detect_outliers(df[x])
        print(f"Number of outliers in column '{x}': {num_outliers}")
    

Number of outliers in column 'step': 102688
Number of outliers in column 'amount': 338078
Number of outliers in column 'oldbalanceOrg': 1112507
Number of outliers in column 'newbalanceOrig': 1053391
Number of outliers in column 'oldbalanceDest': 786135
Number of outliers in column 'newbalanceDest': 738527
Number of outliers in column 'isFraud': 8213
Number of outliers in column 'isFlaggedFraud': 16


In [8]:
df.type.unique()

array(['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN'],
      dtype=object)

In [9]:
from sklearn import preprocessing 
le=preprocessing.LabelEncoder()  
df1['type']=le.fit_transform(df1['type']) 

In [10]:
df1['destination diff']=df1.newbalanceDest-df1.oldbalanceDest
df1['Original diff']=df1.newbalanceOrig-df1.oldbalanceOrg

In [11]:
df1.corr()

C:\Users\Siddhi\AppData\Local\Temp\ipykernel_24364\473017434.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df1.corr()


,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,destination diff,Original diff
step,1.000000,0.006635,0.022373,-0.010058,-0.010299,0.027665,0.025888,0.031578,0.003277,0.001325,-0.007255
type,0.006635,1.000000,0.088419,-0.339760,-0.352758,-0.104679,-0.059364,0.020833,0.002685,0.169399,-0.342153
amount,0.022373,0.088419,1.000000,-0.002762,-0.007861,0.294137,0.459304,0.076688,0.012295,0.845964,-0.102337
oldbalanceOrg,-0.010058,-0.339760,-0.002762,1.000000,0.998803,0.066243,0.042029,0.010154,0.003835,-0.087032,0.220297
newbalanceOrig,-0.010299,-0.352758,-0.007861,0.998803,1.000000,0.067812,0.041837,-0.008148,0.003776,-0.094456,0.267750
oldbalanceDest,0.027665,-0.104679,0.294137,0.066243,0.067812,1.000000,0.976569,-0.005885,-0.000513,0.232316,0.047460
newbalanceDest,0.025888,-0.059364,0.459304,0.042029,0.041837,0.976569,1.000000,0.000535,-0.000529,0.436191,0.006451
isFraud,0.031578,0.020833,0.076688,0.010154,-0.008148,-0.005885,0.000535,1.000000,0.044109,0.027028,-0.362472
isFlaggedFraud,0.003277,0.002685,0.012295,0.003835,0.003776,-0.000513,-0.000529,0.044109,1.000000,-0.000242,-0.000230
destination diff,0.001325,0.169399,0.845964,-0.087032,-0.094456,0.232316,0.436191,0.027028,-0.000242,1.000000,-0.169292


In [12]:
del df1["newbalanceDest"]
del df1["oldbalanceDest"]
del df1["newbalanceOrig"]
del df1["oldbalanceOrg"]


In [13]:
df1.corr()


C:\Users\Siddhi\AppData\Local\Temp\ipykernel_24364\1707261694.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df1.corr()


,step,type,amount,isFraud,isFlaggedFraud,destination diff,Original diff
step,1.000000,0.006635,0.022373,0.031578,0.003277,0.001325,-0.007255
type,0.006635,1.000000,0.088419,0.020833,0.002685,0.169399,-0.342153
amount,0.022373,0.088419,1.000000,0.076688,0.012295,0.845964,-0.102337
isFraud,0.031578,0.020833,0.076688,1.000000,0.044109,0.027028,-0.362472
isFlaggedFraud,0.003277,0.002685,0.012295,0.044109,1.000000,-0.000242,-0.000230
destination diff,0.001325,0.169399,0.845964,0.027028,-0.000242,1.000000,-0.169292
Original diff,-0.007255,-0.342153,-0.102337,-0.362472,-0.000230,-0.169292,1.000000


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report


#since there are many outliers in the important features, I have tried using three ML models which are not as sensitive 
# compared to other models. Removing outliers in cases where there are binary classes will affect the entire prediction 
# and they cannot be exchanged with the mean,median or mode. 


independent=["step","type","amount","isFlaggedFraud","destination diff","Original diff"]

# I have selected the above features for prediction because name of the source/destination does not affect the decision
# of a transaction being fraud or not. Features like total amount transacted, and the resultant balance in original account
# will allow us to understand if its a fraud or not. Even the difference in the balance of the destination account. And many
# times there are cases where the transaction type and how long it took leads to understanding if its a fraud or not. Eg: 
# if a transaction is taking long then the destination must be well hidden.

df_subset = df1.sample(frac=0.1, random_state=42)

#due to large data size, I have tried random sampling to see how it works. Due to need of more storage I shifted to kaggle 
# to use its storage and GPU power. The accuracy remains the same even when applying it to the entire dataset.


X_subset = df_subset[independent]
y_subset = df_subset['isFraud']
X_train_subset, X_test_subset, y_train_subset, y_test_subset = train_test_split(
    X_subset, y_subset, test_size=0.2, random_state=42)


rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_subset, y_train_subset)
rf_predictions = rf_model.predict(X_test_subset)


print("Random Forest:")
print("Accuracy:", accuracy_score(y_test_subset, rf_predictions))
print("Classification Report:\n", classification_report(y_test_subset, rf_predictions))


Random Forest:
Accuracy: 0.9995992235939428
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    127079
           1       0.97      0.73      0.83       174

    accuracy                           1.00    127253
   macro avg       0.98      0.86      0.92    127253
weighted avg       1.00      1.00      1.00    127253



In [15]:


xgb_model = XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train_subset, y_train_subset)
xgb_predictions = xgb_model.predict(X_test_subset)

print("\nXGBoost:")
print("Accuracy:", accuracy_score(y_test_subset, xgb_predictions))
print("Classification Report:\n", classification_report(y_test_subset, xgb_predictions))



XGBoost:
Accuracy: 0.9995677901503305
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    127079
           1       0.96      0.71      0.82       174

    accuracy                           1.00    127253
   macro avg       0.98      0.86      0.91    127253
weighted avg       1.00      1.00      1.00    127253



In [16]:

svm_model = SVC(random_state=42)
svm_model.fit(X_train_subset, y_train_subset)
svm_predictions = svm_model.predict(X_test_subset)

print("\nSVM:")
print("Accuracy:", accuracy_score(y_test_subset, svm_predictions))
print("Classification Report:\n", classification_report(y_test_subset, svm_predictions))


SVM:
Accuracy: 0.9990805717743393
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    127079
           1       0.91      0.36      0.52       174

    accuracy                           1.00    127253
   macro avg       0.96      0.68      0.76    127253
weighted avg       1.00      1.00      1.00    127253

